<a href="https://colab.research.google.com/github/putrareddy/Abstractive-Summarizer/blob/main/Abstractive_Summarizer!!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing Required Dependencies and Models**

In [1]:
!pip install pytorch_transformers
!pip install transformers
!pip install datasets
!pip install nltk
from nltk.translate.bleu_score import SmoothingFunction, corpus_bleu, sentence_bleu
#Emotion Detection
!pip install text2emotion
import text2emotion as te
!pip install gtts
from gtts import gTTS #Import Google Text to Speech
from IPython.display import Audio #Import Audio method from IPython's Display Class
import librosa #to get duration of sound
import time #to make program sleep for some time
import nltk
nltk.download('punkt')
import torch
import datasets
import pandas as pd
import spacy
from spacy import displacy
NER = spacy.load("en_core_web_sm")
import tqdm
from tqdm.auto import tqdm
import csv
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
import tensorflow as tf

from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import BertForQuestionAnswering, BertTokenizer
from transformers import ProphetNetTokenizer, ProphetNetForConditionalGeneration, ProphetNetConfig

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 47.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 94.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 8.4 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=42667c6f23b8da35fd66097af9e5caf2436d1dc85b5aae32b6501d3e225b8c1d
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def split(text):
  splitted = nltk.tokenize.sent_tokenize(text)
  return splitted

In [3]:
def generate_summary(text):
  # set seed to reproduce results. Feel free to change the seed though to get different results
  tf.random.set_seed(0)
  inputs = summary_tokenizer(text, return_tensors='pt', return_attention_mask=True, truncation=True, padding=True, return_token_type_ids=True)
  summary_ids = summary_model.generate(inputs["input_ids"], num_beams=5, top_k=50, top_p=0.95, max_length=100, length_penalty=1.0, repetition_penalty=2.5, no_repeat_ngram_size=3)
  summary_txt = summary_tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)
  return summary_txt

In [4]:
def question_generation(splitted):
  questions = []
  for sentence in splitted:
    inputs = question_tokenizer([sentence], return_tensors='pt')
    # Generate Questions
    question_ids = question_model.generate(inputs['input_ids'], num_beams=5, early_stopping=True)
    ques=question_tokenizer.batch_decode(question_ids, skip_special_tokens=True)
    questions.append(ques[0])
  return questions

In [5]:
def answer_generation(question, answer_text):
    '''
    Takes a `question` string and an `answer_text` string (which contains the
    answer), and identifies the words within the `answer_text` that are the
    answer. Prints them out.
    '''
    # ======== Tokenize ========
    # Apply the tokenizer to the input text, treating them as a text-pair.
    input_ids = answer_tokenizer.encode(question, answer_text, truncation=True)

    # Report how long the input sequence is.
    #print('Query has {:,} tokens.\n'.format(len(input_ids)))

    # ======== Set Segment IDs ========
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(answer_tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # ======== Evaluate ========
    # Run our example question through the model.
    start_scores, end_scores = answer_model(torch.tensor([input_ids]), # The tokens representing our input text.
                                    token_type_ids=torch.tensor([segment_ids])) # The segment IDs to differentiate question from answer_text

    # ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Get the string versions of the input tokens.
    tokens = answer_tokenizer.convert_ids_to_tokens(input_ids)

    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):

        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]

        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]
    #print("Question: " + question)
    #print('Answer: "' + answer + '"')
    return answer

In [6]:
def answering(questions,source,summary):
  answer_source = []
  answer_summary = []
  for question in questions:
    answer_source.append(answer_generation(question, source))
    answer_summary.append(answer_generation(question, summary))
  return answer_source,answer_summary

In [7]:
def entity(text):
  sample_text = NER(text)
  for word in sample_text.ents:
    return word.label

In [8]:
def entities_count(entities_article,entities_summary):
  l1 = list(j.text for j in entities_article.ents)
  l2 = list(j.text for j in entities_summary.ents)
  count = 0
  for j in l2:
    if j in l1:
      count = count+1
  perc = (count/(len(l2)))*100
  # perc = str(perc)+'%'
  count = str(count) + " Entities"
  return count,perc

In [9]:
def replacing(answer_source,answer_summary,summary):
  for j in range(len(answer_summary)):
    if '[CLS]' and '[SEP]' not in answer_source[j]:
      if answer_source[j] != answer_summary[j]:
        if answer_source[j] not in summary:
          if " " in answer_source[j] or entity(answer_source[j]) == entity(answer_summary[j]):
            summary = summary.replace(answer_summary[j],answer_source[j])
  return summary

In [10]:
answer_model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad',return_dict=False)
answer_tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [11]:
question_model = ProphetNetForConditionalGeneration.from_pretrained('microsoft/prophetnet-large-uncased-squad-qg')
question_tokenizer = ProphetNetTokenizer.from_pretrained('microsoft/prophetnet-large-uncased-squad-qg')

In [12]:
summary_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
summary_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

# **Importing Dataset**

In [13]:
all_ds=datasets.list_datasets()
'cnn_dailymail' in all_ds

<ipython-input-13-24a52a914fac>:1: FutureWarning: list_datasets is deprecated and will be removed in the next major version of datasets. Use 'huggingface_hub.list_datasets' instead.
  all_ds=datasets.list_datasets()


True

In [14]:
'xsum' in all_ds

True

In [15]:
cnndm_dataset = datasets.load_dataset('cnn_dailymail', '3.0.0')
xsum_dataset = datasets.load_dataset('xsum')

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [16]:
df = pd.DataFrame(columns = ["Article", "Gold Summary"])

articles = []

for sample in tqdm(cnndm_dataset['train']):
    # remove newline characters from each sample as we need to use exclusively as seperators
    sample = sample['article'].split('\n')
    articles.append(sample)


goldsummaries = []
for sample in tqdm(cnndm_dataset['train']):
    # remove newline characters from each sample as we need to use exclusively as seperators
    sample = sample['highlights'].split('\n')
    goldsummaries.append(sample)

indices = []
for sample in tqdm(cnndm_dataset['train']):
    # remove newline characters from each sample as we need to use exclusively as seperators
    sample = sample['id'].split('\n')
    indices.append(sample)

xsum_articles = []
for sample in tqdm(xsum_dataset['train']['document']):
  xsum_articles.append(sample)

xsum_goldsummaries = []
for sample in tqdm(xsum_dataset['train']['summary']):
  xsum_goldsummaries.append(sample)

xsum_indices = []
for sample in tqdm(xsum_dataset['train']['id']):
  xsum_indices.append(sample)

  0%|          | 0/287113 [00:00<?, ?it/s]

  0%|          | 0/287113 [00:00<?, ?it/s]

  0%|          | 0/287113 [00:00<?, ?it/s]

  0%|          | 0/204045 [00:00<?, ?it/s]

  0%|          | 0/204045 [00:00<?, ?it/s]

  0%|          | 0/204045 [00:00<?, ?it/s]

In [17]:
df = pd.DataFrame(zip(indices, articles, goldsummaries), columns=['Id','Article','Gold Summary'])
df['Generated Summary'] = ''
df['Number of Common Entities'] = ''
df['% of Entities of Summary in Article'] = ''


xsum_df = pd.DataFrame(zip(xsum_indices, xsum_articles, xsum_goldsummaries), columns=['Id','Article','Gold Summary'])
xsum_df['Generated Summary'] = ''
xsum_df['Number of Common Entities'] = ''
xsum_df['% of Entities of Summary in Article'] = ''

#Changing ele of df from lists to string
df['Article'] = df['Article'].apply(lambda x: ','.join(map(str, x)))
df['Gold Summary'] = df['Gold Summary'].apply(lambda x: ','.join(map(str, x)))
df['Id'] = df['Id'].apply(lambda x: ','.join(map(str, x)))

In [18]:
df

,Id,Article,Gold Summary,Generated Summary,Number of Common Entities,% of Entities of Summary in Article
0,42c027e4ff9730fbb3de84c1af0d2c506e41c3e4,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...,,,
1,ee8871b15c50d0db17b0179a6d2beab35065f1e9,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...,,,
2,06352019a19ae31e527f37f7571c6dd7f0c5da37,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa...",,,
3,24521a2abb2e1f5e34e6824e0f9e56904a2b0e88,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non...",,,
4,7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical...",,,
...,...,...,...,...,...,...
287108,279a12d3ee37b8109cc192a9e88115a5a631fb06,"The nine-year-old daughter of a black, unarmed...","Rumain Brisbon, 34, was killed after Phoenix p...",,,
287109,b5bc9d404a9a5d890c9fc26550b67e6d8d83241f,Legalising assisted suicide is a slippery slop...,"Theo Boer, a European assisted suicide watchdo...",,,
287110,500862586f925e406f8b662934e1a71bbee32463,A group calling itself 'The Women of the 99 Pe...,Ohio congressman criticised for 'condoning the...,,,
287111,32a1f9e5c37a938c0c0bca1a1559247b9c4334b2,Most men enjoy a good pint of lager or real al...,The Black Country Ale Tairsters have been to 1...,,,


In [19]:
xsum_df

,Id,Article,Gold Summary,Generated Summary,Number of Common Entities,% of Entities of Summary in Article
0,35232142,"The full cost of damage in Newton Stewart, one...",Clean-up operations are continuing across the ...,,,
1,40143035,A fire alarm went off at the Holiday Inn in Ho...,Two tourist buses have been destroyed by fire ...,,,
2,35951548,Ferrari appeared in a position to challenge un...,Lewis Hamilton stormed to pole position at the...,,,
3,36266422,"John Edward Bates, formerly of Spalding, Linco...",A former Lincolnshire Police officer carried o...,,,
4,38826984,Patients and staff were evacuated from Cerahpa...,An armed man who locked himself into a room at...,,,
...,...,...,...,...,...,...
204040,34084759,The initial figure released in July was booste...,UK economic growth for the second quarter of t...,,,
204041,40552318,"MEPs, including European Parliament chief Brex...",Theresa May's offer to give EU citizens in the...,,,
204042,36781065,Lincoln Red Imps will bring a 1-0 lead to Glas...,Erik Sviatchenko is adamant that Celtic will p...,,,
204043,31579588,Former Liverpool defender Mark Lawrenson expan...,People have spent a large part of this season ...,,,


In [20]:
for i in tqdm(range(len(df[:10]))):
  text = df['Article'][i]
  #Generating Summary for Input CNN_DM Article
  summary_txt = generate_summary(text)
  df['Generated Summary'][i] = summary_txt
  entities_article = NER(df['Article'][i])
  entities_summary = NER(df['Generated Summary'][i])
  count,perc = entities_count(entities_article,entities_summary)
  df['% of Entities of Summary in Article'][i] = perc
  df['Number of Common Entities'][i] = count

initial_df = df.copy()
initial_df[:10]

  0%|          | 0/10 [00:00<?, ?it/s]

,Id,Article,Gold Summary,Generated Summary,Number of Common Entities,% of Entities of Summary in Article
0,42c027e4ff9730fbb3de84c1af0d2c506e41c3e4,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...,Harry Potter star Daniel Radcliffe turns 18 on...,6 Entities,100.0
1,ee8871b15c50d0db17b0179a6d2beab35065f1e9,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...,Judge Steven Leifman is an advocate for justic...,3 Entities,75.0
2,06352019a19ae31e527f37f7571c6dd7f0c5da37,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa...","""I probably had a 30-, 35-foot free fall,"" sur...",7 Entities,100.0
3,24521a2abb2e1f5e34e6824e0f9e56904a2b0e88,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non...",Doctors remove five small polyps from Presiden...,6 Entities,100.0
4,7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical...",NEW: Atlanta Falcons owner Arthur Blank calls ...,5 Entities,100.0
5,a1ebb8bb4d370a1fdf28769206d572be60642d70,"BAGHDAD, Iraq (CNN) -- Dressed in a Superman s...","Parents beam with pride, can't stop from smili...","""I was so happy I didn't know what to do with ...",2 Entities,100.0
6,7c0e61ac829a3b3b653e2e3e7536cc4881d1f264,"BAGHDAD, Iraq (CNN) -- The women are too afrai...","Aid workers: Violence, increased cost of livin...",Women say they are driven to prostitution by a...,1 Entities,100.0
7,f0d73bdab711763e745cdc75850861c9018f235d,"BOGOTA, Colombia (CNN) -- A key rebel commande...",Tomas Medina Caracas was a fugitive from a U.S...,"Tomas Medina Caracas, known popularly as ""El N...",6 Entities,100.0
8,5e22bbfc7232418b8d2dd646b952e404df5bd048,WASHINGTON (CNN) -- White House press secretar...,"President Bush says Tony Snow ""will battle can...",White House press secretary Tony Snow to step ...,5 Entities,100.0
9,613d6311ec2c1985bd44707d1796d275452fe156,(CNN) -- Police and FBI agents are investigati...,Empty anti-tank weapon turns up in front of Ne...,20-year-old AT4 anti-tank rocket launcher tube...,6 Entities,85.714286


In [21]:
for i in tqdm(range(len(xsum_df[:10]))):
  text = xsum_df['Article'][i]
  #Generating Summary for Input XSum Article
  summary_txt = generate_summary(text)
  xsum_df['Generated Summary'][i] = summary_txt
  entities_article = NER(xsum_df['Article'][i])
  entities_summary = NER(xsum_df['Generated Summary'][i])
  count,perc = entities_count(entities_article,entities_summary)
  xsum_df['% of Entities of Summary in Article'][i] = perc
  xsum_df['Number of Common Entities'][i] = count

initial_xsum_df = xsum_df.copy()
initial_xsum_df[:10]

  0%|          | 0/10 [00:00<?, ?it/s]

,Id,Article,Gold Summary,Generated Summary,Number of Common Entities,% of Entities of Summary in Article
0,35232142,"The full cost of damage in Newton Stewart, one...",Clean-up operations are continuing across the ...,"The full cost of damage in Newton Stewart, one...",8 Entities,88.888889
1,40143035,A fire alarm went off at the Holiday Inn in Ho...,Two tourist buses have been destroyed by fire ...,A fire alarm went off at the Holiday Inn in Ho...,6 Entities,100.0
2,35951548,Ferrari appeared in a position to challenge un...,Lewis Hamilton stormed to pole position at the...,Ferrari appeared in a position to challenge un...,10 Entities,100.0
3,36266422,"John Edward Bates, formerly of Spalding, Linco...",A former Lincolnshire Police officer carried o...,"John Edward Bates faces a total of 22 charges,...",9 Entities,100.0
4,38826984,Patients and staff were evacuated from Cerahpa...,An armed man who locked himself into a room at...,Patients and staff were evacuated from Cerahpa...,2 Entities,100.0
5,34540833,Simone Favaro got the crucial try with the las...,Defending Pro12 champions Glasgow Warriors bag...,Simone Favaro got the crucial try with the las...,7 Entities,100.0
6,20836172,"Veronica Vanessa Chango-Alverez, 31, was kille...",A man with links to a car that was involved in...,"Veronica Vanessa Chango-Alverez, 31, was kille...",8 Entities,100.0
7,35932467,Belgian cyclist Demoitie died after a collisio...,Welsh cyclist Luke Rowe says changes to the sp...,Demoitie died after a collision with a motorbi...,5 Entities,71.428571
8,40758845,"Gundogan, 26, told BBC Sport he ""can see the f...",Manchester City midfielder Ilkay Gundogan says...,Gundogan will not be fit for the start of the ...,13 Entities,100.0
9,30358490,The crash happened about 07:20 GMT at the junc...,A jogger has been hit by an unmarked police ca...,The crash happened at the junction of the A127...,3 Entities,100.0


# **Question Generation**

In [22]:
for i in tqdm(range(len(df[:10]))):
  source = df["Article"][i]
  summary = df['Generated Summary'][i]
  print("Initial :" + summary)
  splitted_summary = split(summary)
  #print(splitted_summary)
  questions = question_generation(splitted_summary)
  #print(questions)
  answer_source,answer_summary = answering(questions, source, summary)
  summary = replacing(answer_source,answer_summary,summary)
  for j in range(len(answer_summary)):
    if '[CLS]' and '[SEP]' not in answer_source[j]:
      if answer_source[j] != answer_summary[j]:
        if answer_source[j] not in summary:
          if " " in answer_source[j] or entity(answer_source[j]) == entity(answer_summary[j]):
            summary = summary.replace(answer_summary[j],answer_source[j])

  df["Generated Summary"][i] = summary
  print("Final : "+ summary)
  print(answer_source)
  print(answer_summary)
  print("\n")

final_df = df.copy()
df[:10]

  0%|          | 0/10 [00:00<?, ?it/s]

Initial :Harry Potter star Daniel Radcliffe turns 18 on Monday. He gains access to a reported £20 million ($41.1 million) fortune. The young actor says he has no plans to fritter his cash away. Details of how he'll mark his landmark birthday are under wraps.


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if s

Final : Harry Potter star Daniel Radcliffe turns 18 on Monday. He gains access to a reported £20 million ($41.1 million) fortune. The young actor says he has no plans to fritter his cash away. Details of how he'll mark his landmark birthday are have some sort of party.
['monday', '£20 million ( $ 41 . 1 million ) fortune', 'no', 'have some sort of party']
['monday', 'a reported £20 million ( $ 41 . 1 million ) fortune', 'no', 'under wraps']


Initial :Judge Steven Leifman is an advocate for justice and the mentally ill. About one-third of all people in Miami-Dade county jails are mentally ill, he says. In 2008, many inmates who would otherwise have been brought to "forgotten floor" will be sent to a new mental health facility.


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


...

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Final : Women say they are driven to prostitution by sell their bodies to put food on the table for their children. Violence, increased cost of living, lack of government aid leave women with few other options. "I have to do anything that I can to preserve my child, because I am a mother," one woman says. Activist: "There is a huge population of women who have to sell their bodies"
['sell their bodies to put food on the table for their children', 'prostitution', 'anything that i can', '[SEP]']
['a desperate desire for survival', 'violence , increased cost of living , lack of government aid', 'anything that i can', 'activist']


Initial :Tomas Medina Caracas, known popularly as "El Negro Acacio," killed in Colombia. He was a member of the high command of the Fuerzas Armadas Revolucionarias de Colombia. U.S. officials alleged he helped manage the group's extensive cocaine trafficking network.
Final : Tomas Medina Caracas, known popularly as "El Negro Acacio," killed in Colombia. He was a

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Final : White House press secretary Tony Snow to step down on September 14. He will be replaced by deputy press secretary Dana Perino. President Bush says he will "sadly accept" Snow's resignation. Snow: "This job has really been a dream for me, a blast"
['september 14', 'deputy press secretary dana perino', 'he will " sadly accept "', 'he would continue to speak out on issues , and would do " some radio , some tv , but i don \' t anticipate full - time anchor duties . "']
['september 14', 'dana perino', 'he will " sadly accept "', 'this job has really been a dream for me , a blast "']


Initial :20-year-old AT4 anti-tank rocket launcher tube found on woman's lawn. Launcher has been turned over to U.S. Army officials at Fort Monmouth, New Jersey. No indication launcher had been fired recently, police say. "Al Qaeda doesn't leave a rocket launcher on the lawn of middle-aged ladies"
Final : 20-year-old AT4 anti-tank rocket launcher tube found on woman's lawn. Launcher has been turned ove

,Id,Article,Gold Summary,Generated Summary,Number of Common Entities,% of Entities of Summary in Article
0,42c027e4ff9730fbb3de84c1af0d2c506e41c3e4,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...,Harry Potter star Daniel Radcliffe turns 18 on...,6 Entities,100.0
1,ee8871b15c50d0db17b0179a6d2beab35065f1e9,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...,Judge Steven Leifman is well known in miami as...,3 Entities,75.0
2,06352019a19ae31e527f37f7571c6dd7f0c5da37,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa...","""I probably had a 30-, 35-foot free fall,"" sur...",7 Entities,100.0
3,24521a2abb2e1f5e34e6824e0f9e56904a2b0e88,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non...",Doctors remove five small polyps from Presiden...,6 Entities,100.0
4,7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical...",NEW: Atlanta Falcons owner Arthur Blank calls ...,5 Entities,100.0
5,a1ebb8bb4d370a1fdf28769206d572be60642d70,"BAGHDAD, Iraq (CNN) -- Dressed in a Superman s...","Parents beam with pride, can't stop from smili...","""I was so a broad smile I didn't know what to ...",2 Entities,100.0
6,7c0e61ac829a3b3b653e2e3e7536cc4881d1f264,"BAGHDAD, Iraq (CNN) -- The women are too afrai...","Aid workers: Violence, increased cost of livin...",Women say they are driven to prostitution by s...,1 Entities,100.0
7,f0d73bdab711763e745cdc75850861c9018f235d,"BOGOTA, Colombia (CNN) -- A key rebel commande...",Tomas Medina Caracas was a fugitive from a U.S...,"Tomas Medina Caracas, known popularly as ""El N...",6 Entities,100.0
8,5e22bbfc7232418b8d2dd646b952e404df5bd048,WASHINGTON (CNN) -- White House press secretar...,"President Bush says Tony Snow ""will battle can...",White House press secretary Tony Snow to step ...,5 Entities,100.0
9,613d6311ec2c1985bd44707d1796d275452fe156,(CNN) -- Police and FBI agents are investigati...,Empty anti-tank weapon turns up in front of Ne...,20-year-old AT4 anti-tank rocket launcher tube...,6 Entities,85.714286


In [23]:
for i in tqdm(range(len(xsum_df[:10]))):
  source = xsum_df["Article"][i]
  summary = xsum_df["Generated Summary"][i]
  print("Initial :" + summary)
  splitted_summary = split(summary)
  #print(splitted_summary)
  questions = question_generation(splitted_summary)
  #print(questions)
  answer_source,answer_summary = answering(questions, source, summary)
  summary = replacing(answer_source,answer_summary,summary)
  '''for j in range(len(answer_summary)):
    if '[CLS]' and '[SEP]' not in answer_source[j]:
      if answer_source[j] != answer_summary[j]:
        if answer_source[j] not in summary:
          if " " in answer_source[j] or entity(answer_source[j]) == entity(answer_summary[j]):
            summary = summary.replace(answer_summary[j],answer_source[j])'''

  xsum_df["Generated Summary"][i] = summary
  print("Final : "+ summary)
  print(answer_source)
  print(answer_summary)
  print("\n")

final_xsum_df = xsum_df.copy()
xsum_df[:10]

  0%|          | 0/10 [00:00<?, ?it/s]

Initial :The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed. Flood alert remains in place across the Borders because of the constant rain. Labour Party's deputy Scottish leader Alex Rowley was in Hawick on Monday to see the situation first hand.


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Final : The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed. Flood alert remains in place across the Borders because of the constant rain. Labour Party's deputy Scottish leader Alex Rowley was in Hawick on Monday to see the situation first hand.
['the full cost of damage in newton stewart , one of the areas worst affected , is still being assessed', 'because of the constant rain', 'alex rowley']
['the full cost of damage in newton stewart , one of the areas worst affected , is still being assessed', 'the constant rain', 'alex rowley']


Initial :A fire alarm went off at the Holiday Inn in Hope Street at about 04:20 BST. One of the tour groups is from Germany, the other from China and Taiwan. The driver of one of the buses said many of the passengers had left personal belongings on board and these had been destroyed.
Final : A fire alarm went off at the Holiday Inn in Hope Street at about 04:20 BST. One of the tour groups is from Germany, 

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


...

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Final : Simone Favaro got the crucial try with the last move of the game. Earlier touchdowns by Chris Fusaro, Zander Fagerson and Junior Bulumakau. Rynard Landman and Ashton Hewitt got a try in either half for the Dragons.
['simone favaro', '[SEP]', 'rynard landman and ashton hewitt']
['simone favaro', 'zander fagerson', 'rynard landman and ashton hewitt']


Initial :Veronica Vanessa Chango-Alverez, 31, was killed and another man injured. Audi A3 struck them in Streatham High Road at 05:30 GMT on Saturday. Police want to trace Nathan Davis, 27, who they say has links to the Audi.
Final : Veronica Vanessa Chango-Alverez, 31, was killed and another man injured. Audi A3 struck them in Streatham High Road at 05:30 GMT on Saturday. Police want to trace Nathan Davis, 27, who they say has links to the Audi.
['killed and another man injured', '05 : 30 gmt on saturday', 'police want to trace nathan davis , 27 , who they say has links to the audi']
['killed and another man injured', 'saturday', 

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Final : Gundogan will not be fit for the start of the Premier League season at Brighton on 12 August. The German missed the 2014 World Cup following back surgery that kept him out for a year, and sat out Euro 2016 because of a dislocated kneecap. Gundogan made 15 appearances and scored five goals in his debut season for City following his £20m move from Borussia Dortmund.
['gundogan will not be fit', 'a dislocated kneecap', '15']
['gundogan will not be fit', 'a dislocated kneecap', '15']


Initial :The crash happened at the junction of the A127 and Progress Road in Leigh-on-Sea. The man, who police said is aged in his 20s, was treated at the scene for a head injury. He was airlifted to the Royal London Hospital for further treatment.
Final : The crash happened at the junction of the A127 and Progress Road in Leigh-on-Sea. The man, who police said is aged in his 20s, was treated at the scene for a head injury. He was airlifted to the Royal London Hospital for further treatment.
['leigh 

,Id,Article,Gold Summary,Generated Summary,Number of Common Entities,% of Entities of Summary in Article
0,35232142,"The full cost of damage in Newton Stewart, one...",Clean-up operations are continuing across the ...,"The full cost of damage in Newton Stewart, one...",8 Entities,88.888889
1,40143035,A fire alarm went off at the Holiday Inn in Ho...,Two tourist buses have been destroyed by fire ...,A fire alarm went off at the Holiday Inn in Ho...,6 Entities,100.0
2,35951548,Ferrari appeared in a position to challenge un...,Lewis Hamilton stormed to pole position at the...,Ferrari appeared in a position to challenge un...,10 Entities,100.0
3,36266422,"John Edward Bates, formerly of Spalding, Linco...",A former Lincolnshire Police officer carried o...,"John Edward Bates faces a total of 22 charges,...",9 Entities,100.0
4,38826984,Patients and staff were evacuated from Cerahpa...,An armed man who locked himself into a room at...,Patients and staff were evacuated from Cerahpa...,2 Entities,100.0
5,34540833,Simone Favaro got the crucial try with the las...,Defending Pro12 champions Glasgow Warriors bag...,Simone Favaro got the crucial try with the las...,7 Entities,100.0
6,20836172,"Veronica Vanessa Chango-Alverez, 31, was kille...",A man with links to a car that was involved in...,"Veronica Vanessa Chango-Alverez, 31, was kille...",8 Entities,100.0
7,35932467,Belgian cyclist Demoitie died after a collisio...,Welsh cyclist Luke Rowe says changes to the sp...,Demoitie died after a collision with a motorbi...,5 Entities,71.428571
8,40758845,"Gundogan, 26, told BBC Sport he ""can see the f...",Manchester City midfielder Ilkay Gundogan says...,Gundogan will not be fit for the start of the ...,13 Entities,100.0
9,30358490,The crash happened about 07:20 GMT at the junc...,A jogger has been hit by an unmarked police ca...,The crash happened at the junction of the A127...,3 Entities,100.0


In [24]:
for i in tqdm(range(len(df[:10]))):
  entities_article = NER(df['Article'][i])
  entities_summary = NER(df['Generated Summary'][i])
  count,perc = entities_count(entities_article,entities_summary)
  df['% of Entities of Summary in Article'][i] = perc
  df['Number of Common Entities'][i] = count

final_df = df.copy()
final_df[:10]

  0%|          | 0/10 [00:00<?, ?it/s]

,Id,Article,Gold Summary,Generated Summary,Number of Common Entities,% of Entities of Summary in Article
0,42c027e4ff9730fbb3de84c1af0d2c506e41c3e4,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...,Harry Potter star Daniel Radcliffe turns 18 on...,6 Entities,100.0
1,ee8871b15c50d0db17b0179a6d2beab35065f1e9,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...,Judge Steven Leifman is well known in miami as...,3 Entities,60.0
2,06352019a19ae31e527f37f7571c6dd7f0c5da37,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa...","""I probably had a 30-, 35-foot free fall,"" sur...",7 Entities,100.0
3,24521a2abb2e1f5e34e6824e0f9e56904a2b0e88,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non...",Doctors remove five small polyps from Presiden...,6 Entities,85.714286
4,7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical...",NEW: Atlanta Falcons owner Arthur Blank calls ...,5 Entities,100.0
5,a1ebb8bb4d370a1fdf28769206d572be60642d70,"BAGHDAD, Iraq (CNN) -- Dressed in a Superman s...","Parents beam with pride, can't stop from smili...","""I was so a broad smile I didn't know what to ...",2 Entities,100.0
6,7c0e61ac829a3b3b653e2e3e7536cc4881d1f264,"BAGHDAD, Iraq (CNN) -- The women are too afrai...","Aid workers: Violence, increased cost of livin...",Women say they are driven to prostitution by s...,1 Entities,100.0
7,f0d73bdab711763e745cdc75850861c9018f235d,"BOGOTA, Colombia (CNN) -- A key rebel commande...",Tomas Medina Caracas was a fugitive from a U.S...,"Tomas Medina Caracas, known popularly as ""El N...",6 Entities,100.0
8,5e22bbfc7232418b8d2dd646b952e404df5bd048,WASHINGTON (CNN) -- White House press secretar...,"President Bush says Tony Snow ""will battle can...",White House press secretary Tony Snow to step ...,5 Entities,100.0
9,613d6311ec2c1985bd44707d1796d275452fe156,(CNN) -- Police and FBI agents are investigati...,Empty anti-tank weapon turns up in front of Ne...,20-year-old AT4 anti-tank rocket launcher tube...,6 Entities,85.714286


In [25]:
for i in tqdm(range(len(xsum_df[:10]))):
  entities_article = NER(xsum_df['Article'][i])
  entities_summary = NER(xsum_df['Generated Summary'][i])
  count,perc = entities_count(entities_article,entities_summary)
  xsum_df['% of Entities of Summary in Article'][i] = perc
  xsum_df['Number of Common Entities'][i] = count

final_xsum_df = xsum_df.copy()
final_xsum_df[:10]

  0%|          | 0/10 [00:00<?, ?it/s]

,Id,Article,Gold Summary,Generated Summary,Number of Common Entities,% of Entities of Summary in Article
0,35232142,"The full cost of damage in Newton Stewart, one...",Clean-up operations are continuing across the ...,"The full cost of damage in Newton Stewart, one...",8 Entities,88.888889
1,40143035,A fire alarm went off at the Holiday Inn in Ho...,Two tourist buses have been destroyed by fire ...,A fire alarm went off at the Holiday Inn in Ho...,6 Entities,100.0
2,35951548,Ferrari appeared in a position to challenge un...,Lewis Hamilton stormed to pole position at the...,Ferrari appeared in a position to challenge un...,10 Entities,100.0
3,36266422,"John Edward Bates, formerly of Spalding, Linco...",A former Lincolnshire Police officer carried o...,"John Edward Bates faces a total of 22 charges,...",9 Entities,100.0
4,38826984,Patients and staff were evacuated from Cerahpa...,An armed man who locked himself into a room at...,Patients and staff were evacuated from Cerahpa...,2 Entities,100.0
5,34540833,Simone Favaro got the crucial try with the las...,Defending Pro12 champions Glasgow Warriors bag...,Simone Favaro got the crucial try with the las...,7 Entities,100.0
6,20836172,"Veronica Vanessa Chango-Alverez, 31, was kille...",A man with links to a car that was involved in...,"Veronica Vanessa Chango-Alverez, 31, was kille...",8 Entities,100.0
7,35932467,Belgian cyclist Demoitie died after a collisio...,Welsh cyclist Luke Rowe says changes to the sp...,Demoitie died after a collision with a motorbi...,5 Entities,71.428571
8,40758845,"Gundogan, 26, told BBC Sport he ""can see the f...",Manchester City midfielder Ilkay Gundogan says...,Gundogan will not be fit for the start of the ...,13 Entities,100.0
9,30358490,The crash happened about 07:20 GMT at the junc...,A jogger has been hit by an unmarked police ca...,The crash happened at the junction of the A127...,3 Entities,100.0


In [26]:
df[:10]

,Id,Article,Gold Summary,Generated Summary,Number of Common Entities,% of Entities of Summary in Article
0,42c027e4ff9730fbb3de84c1af0d2c506e41c3e4,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...,Harry Potter star Daniel Radcliffe turns 18 on...,6 Entities,100.0
1,ee8871b15c50d0db17b0179a6d2beab35065f1e9,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...,Judge Steven Leifman is well known in miami as...,3 Entities,60.0
2,06352019a19ae31e527f37f7571c6dd7f0c5da37,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa...","""I probably had a 30-, 35-foot free fall,"" sur...",7 Entities,100.0
3,24521a2abb2e1f5e34e6824e0f9e56904a2b0e88,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non...",Doctors remove five small polyps from Presiden...,6 Entities,85.714286
4,7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical...",NEW: Atlanta Falcons owner Arthur Blank calls ...,5 Entities,100.0
5,a1ebb8bb4d370a1fdf28769206d572be60642d70,"BAGHDAD, Iraq (CNN) -- Dressed in a Superman s...","Parents beam with pride, can't stop from smili...","""I was so a broad smile I didn't know what to ...",2 Entities,100.0
6,7c0e61ac829a3b3b653e2e3e7536cc4881d1f264,"BAGHDAD, Iraq (CNN) -- The women are too afrai...","Aid workers: Violence, increased cost of livin...",Women say they are driven to prostitution by s...,1 Entities,100.0
7,f0d73bdab711763e745cdc75850861c9018f235d,"BOGOTA, Colombia (CNN) -- A key rebel commande...",Tomas Medina Caracas was a fugitive from a U.S...,"Tomas Medina Caracas, known popularly as ""El N...",6 Entities,100.0
8,5e22bbfc7232418b8d2dd646b952e404df5bd048,WASHINGTON (CNN) -- White House press secretar...,"President Bush says Tony Snow ""will battle can...",White House press secretary Tony Snow to step ...,5 Entities,100.0
9,613d6311ec2c1985bd44707d1796d275452fe156,(CNN) -- Police and FBI agents are investigati...,Empty anti-tank weapon turns up in front of Ne...,20-year-old AT4 anti-tank rocket launcher tube...,6 Entities,85.714286


In [27]:
xsum_df[:10]

,Id,Article,Gold Summary,Generated Summary,Number of Common Entities,% of Entities of Summary in Article
0,35232142,"The full cost of damage in Newton Stewart, one...",Clean-up operations are continuing across the ...,"The full cost of damage in Newton Stewart, one...",8 Entities,88.888889
1,40143035,A fire alarm went off at the Holiday Inn in Ho...,Two tourist buses have been destroyed by fire ...,A fire alarm went off at the Holiday Inn in Ho...,6 Entities,100.0
2,35951548,Ferrari appeared in a position to challenge un...,Lewis Hamilton stormed to pole position at the...,Ferrari appeared in a position to challenge un...,10 Entities,100.0
3,36266422,"John Edward Bates, formerly of Spalding, Linco...",A former Lincolnshire Police officer carried o...,"John Edward Bates faces a total of 22 charges,...",9 Entities,100.0
4,38826984,Patients and staff were evacuated from Cerahpa...,An armed man who locked himself into a room at...,Patients and staff were evacuated from Cerahpa...,2 Entities,100.0
5,34540833,Simone Favaro got the crucial try with the las...,Defending Pro12 champions Glasgow Warriors bag...,Simone Favaro got the crucial try with the las...,7 Entities,100.0
6,20836172,"Veronica Vanessa Chango-Alverez, 31, was kille...",A man with links to a car that was involved in...,"Veronica Vanessa Chango-Alverez, 31, was kille...",8 Entities,100.0
7,35932467,Belgian cyclist Demoitie died after a collisio...,Welsh cyclist Luke Rowe says changes to the sp...,Demoitie died after a collision with a motorbi...,5 Entities,71.428571
8,40758845,"Gundogan, 26, told BBC Sport he ""can see the f...",Manchester City midfielder Ilkay Gundogan says...,Gundogan will not be fit for the start of the ...,13 Entities,100.0
9,30358490,The crash happened about 07:20 GMT at the junc...,A jogger has been hit by an unmarked police ca...,The crash happened at the junction of the A127...,3 Entities,100.0


In [28]:
initial_df[:10]

,Id,Article,Gold Summary,Generated Summary,Number of Common Entities,% of Entities of Summary in Article
0,42c027e4ff9730fbb3de84c1af0d2c506e41c3e4,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...,Harry Potter star Daniel Radcliffe turns 18 on...,6 Entities,100.0
1,ee8871b15c50d0db17b0179a6d2beab35065f1e9,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...,Judge Steven Leifman is an advocate for justic...,3 Entities,75.0
2,06352019a19ae31e527f37f7571c6dd7f0c5da37,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa...","""I probably had a 30-, 35-foot free fall,"" sur...",7 Entities,100.0
3,24521a2abb2e1f5e34e6824e0f9e56904a2b0e88,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non...",Doctors remove five small polyps from Presiden...,6 Entities,100.0
4,7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical...",NEW: Atlanta Falcons owner Arthur Blank calls ...,5 Entities,100.0
5,a1ebb8bb4d370a1fdf28769206d572be60642d70,"BAGHDAD, Iraq (CNN) -- Dressed in a Superman s...","Parents beam with pride, can't stop from smili...","""I was so happy I didn't know what to do with ...",2 Entities,100.0
6,7c0e61ac829a3b3b653e2e3e7536cc4881d1f264,"BAGHDAD, Iraq (CNN) -- The women are too afrai...","Aid workers: Violence, increased cost of livin...",Women say they are driven to prostitution by a...,1 Entities,100.0
7,f0d73bdab711763e745cdc75850861c9018f235d,"BOGOTA, Colombia (CNN) -- A key rebel commande...",Tomas Medina Caracas was a fugitive from a U.S...,"Tomas Medina Caracas, known popularly as ""El N...",6 Entities,100.0
8,5e22bbfc7232418b8d2dd646b952e404df5bd048,WASHINGTON (CNN) -- White House press secretar...,"President Bush says Tony Snow ""will battle can...",White House press secretary Tony Snow to step ...,5 Entities,100.0
9,613d6311ec2c1985bd44707d1796d275452fe156,(CNN) -- Police and FBI agents are investigati...,Empty anti-tank weapon turns up in front of Ne...,20-year-old AT4 anti-tank rocket launcher tube...,6 Entities,85.714286


In [29]:
final_df[:10]

,Id,Article,Gold Summary,Generated Summary,Number of Common Entities,% of Entities of Summary in Article
0,42c027e4ff9730fbb3de84c1af0d2c506e41c3e4,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...,Harry Potter star Daniel Radcliffe turns 18 on...,6 Entities,100.0
1,ee8871b15c50d0db17b0179a6d2beab35065f1e9,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...,Judge Steven Leifman is well known in miami as...,3 Entities,60.0
2,06352019a19ae31e527f37f7571c6dd7f0c5da37,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa...","""I probably had a 30-, 35-foot free fall,"" sur...",7 Entities,100.0
3,24521a2abb2e1f5e34e6824e0f9e56904a2b0e88,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non...",Doctors remove five small polyps from Presiden...,6 Entities,85.714286
4,7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical...",NEW: Atlanta Falcons owner Arthur Blank calls ...,5 Entities,100.0
5,a1ebb8bb4d370a1fdf28769206d572be60642d70,"BAGHDAD, Iraq (CNN) -- Dressed in a Superman s...","Parents beam with pride, can't stop from smili...","""I was so a broad smile I didn't know what to ...",2 Entities,100.0
6,7c0e61ac829a3b3b653e2e3e7536cc4881d1f264,"BAGHDAD, Iraq (CNN) -- The women are too afrai...","Aid workers: Violence, increased cost of livin...",Women say they are driven to prostitution by s...,1 Entities,100.0
7,f0d73bdab711763e745cdc75850861c9018f235d,"BOGOTA, Colombia (CNN) -- A key rebel commande...",Tomas Medina Caracas was a fugitive from a U.S...,"Tomas Medina Caracas, known popularly as ""El N...",6 Entities,100.0
8,5e22bbfc7232418b8d2dd646b952e404df5bd048,WASHINGTON (CNN) -- White House press secretar...,"President Bush says Tony Snow ""will battle can...",White House press secretary Tony Snow to step ...,5 Entities,100.0
9,613d6311ec2c1985bd44707d1796d275452fe156,(CNN) -- Police and FBI agents are investigati...,Empty anti-tank weapon turns up in front of Ne...,20-year-old AT4 anti-tank rocket launcher tube...,6 Entities,85.714286


In [42]:
final_df[:10]['% of Entities of Summary in Article'].mean()

93.14285714285714

# **Speech and Emotion Detection**

In [31]:
'''notice = gTTS('Enter the index of Generated Summary to read ') #Provide the string to convert to speech
notice.save('notice.wav') #save the string converted to speech as a .wav file
sound_file = 'notice.wav'
duration = librosa.get_duration(filename='notice.wav')
Audio(sound_file, autoplay=True)'''

"notice = gTTS('Enter the index of Generated Summary to read ') #Provide the string to convert to speech\nnotice.save('notice.wav') #save the string converted to speech as a .wav file\nsound_file = 'notice.wav'\nduration = librosa.get_duration(filename='notice.wav')\nAudio(sound_file, autoplay=True)"

In [32]:
'''time.sleep(duration+2)'''

'time.sleep(duration+2)'

In [33]:
# index = int(input('Enter the index :'))
# gen_summary = gTTS(final_xsum_df['Generated Summary'][index])
# gen_summary.save('gen_summary.wav')
# sound_file = 'gen_summary.wav'
# print(final_xsum_df['Generated Summary'][index])
# print(final_xsum_df['Gold Summary'][index])
# print(final_xsum_df['Article'][index])
# duration = librosa.get_duration(filename='gen_summary.wav')
# Audio(sound_file, autoplay=True)

In [34]:
# initial_xsum_df['Generated Summary'][index]

In [35]:
# time.sleep(duration+2)

In [36]:
# emotion = te.get_emotion(final_xsum_df['Generated Summary'][index])
# emotion

In [37]:
# text = "Angry: " + str(int(emotion['Angry']*100)) + "% , Fear: " + str(int(emotion['Fear']*100)) + "% , Happy: "  + str(int(emotion['Happy']*100)) + "% , Sad: "  + str(int(emotion['Sad']*100)) + "% , Surprise: " + str(int(emotion['Surprise']*100)) + "%."
# print(text)
# emotion_text = gTTS(text)
# emotion_text.save('emotion_text.wav')
# sound_file = 'emotion_text.wav'
# duration = librosa.get_duration(filename='emotion_text.wav')
# Audio(sound_file, autoplay=True)

In [38]:
# time.sleep(duration+2)